In [21]:
from zipfile import ZipFile
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms
from PIL import Image

from sklearn.model_selection import train_test_split

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
zip_path = "/content/drive/MyDrive/A-Z Dataset/A_Z Handwritten Data.csv.zip"

In [4]:
with ZipFile(zip_path) as zipfile:
    zipfile.extractall()

In [6]:
csv_path = "/content/A_Z Handwritten Data.csv"

In [7]:
df = pd.read_csv(csv_path)

In [8]:
X = df.drop('0',axis = 1).to_numpy() # axis=1 is to say to drop along the columns
y = df['0'].to_numpy()

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [12]:
del df

In [13]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2)

In [16]:
class Alphabets(Dataset):

  def __init__(self,X,y,transform=None):
    self.X = X
    self.y = y
    self.transform = transform
  
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self,index):

    image = Image.fromarray(((self.X[index]).astype(np.uint8)).reshape(28,28))
    label = self.y[index]

    if self.transform != None:
      image = self.transform(image)
    
    return image,label


In [28]:
transform = transforms.Compose([
         transforms.ToTensor(),
         transforms.RandomAffine(degrees=(-50,50),
                                 translate=(0.2, 0.2),
                                 scale=(0.5, 0.75))                      
])

In [29]:
train_dataset = Alphabets(X_train,y_train,transform)
test_dataset = Alphabets(X_test,y_test,transform)
val_dataset = Alphabets(X_val,y_val,transform)

In [30]:
train_dataset_loader = DataLoader(train_dataset,batch_size=100,shuffle=True)
test_dataset_loader = DataLoader(test_dataset,batch_size=100,shuffle=True)
val_dataset_loader = DataLoader(val_dataset,batch_size=100,shuffle=True)

In [31]:
import torchvision

inputs_batch, labels_batch = next(iter(train_dataset_loader))
grid = torchvision.utils.make_grid(inputs_batch, nrow=40, pad_value=1)
torchvision.utils.save_image(grid, 'inputs_batch_preview.png')